In [50]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
from sqlalchemy import inspect
import sys
import sqlalchemy.ext.declarative as sqld
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
sys.path.append('..')

In [3]:
from modules.transforms import *

In [4]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [5]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [6]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [7]:
pop=pd.read_sql_table('producto1_Covid_19', schema='producto1',con=cnx) 
pop=pop.groupby('Region')['Poblacion'].sum()
popTOT=pop.sum()

In [8]:
popTOT

19458310.0

In [36]:
DEIS

,index,FECHA_DEF,GLOSA_SEXO,EDAD_TIPO,EDAD_CANT,CODIGO_COMUNA_RESIDENCIA,GLOSA_COMUNA_RESIDENCIA,GLOSA_REG_RES,DIAG1,CAPITULO_DIAG1,...,CODIGO_GRUPO_DIAG2,GLOSA_GRUPO_DIAG2,CODIGO_CATEGORIA_DIAG2,GLOSA_CATEGORIA_DIAG2,CODIGO_SUBCATEGORIA_DIAG2,GLOSA_SUBCATEGORIA_DIAG2,Metropolitana,FECHA_STR,COVID,ANO_DEF
0,485743,2020-08-29,Mujer,1,52,13201,Puente Alto,Metropolitana de Santiago,U072,U00-U99,...,None,None,None,None,None,None,Región Metropolitana,29-08-2020,Covid Sospechoso,2020
1,485744,2020-09-17,Mujer,1,97,13101,Santiago,Metropolitana de Santiago,U072,U00-U99,...,None,None,None,None,None,None,Región Metropolitana,17-09-2020,Covid Sospechoso,2020
2,485745,2020-07-11,Hombre,1,88,13119,Maipú,Metropolitana de Santiago,U072,U00-U99,...,None,None,None,None,None,None,Región Metropolitana,11-07-2020,Covid Sospechoso,2020
3,485746,2020-06-23,Hombre,1,67,9201,Angol,De La Araucanía,U072,U00-U99,...,None,None,None,None,None,None,Resto de Chile,23-06-2020,Covid Sospechoso,2020
4,485747,2020-08-24,Hombre,1,76,13101,Santiago,Metropolitana de Santiago,U072,U00-U99,...,None,None,None,None,None,None,Región Metropolitana,24-08-2020,Covid Sospechoso,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98107,485738,2020-07-04,Hombre,1,73,13119,Maipú,Metropolitana de Santiago,U072,U00-U99,...,None,None,None,None,None,None,Región Metropolitana,04-07-2020,Covid Sospechoso,2020
98108,485739,2020-09-24,Hombre,1,85,9201,Angol,De La Araucanía,U072,U00-U99,...,None,None,None,None,None,None,Resto de Chile,24-09-2020,Covid Sospechoso,2020
98109,485740,2020-07-03,Mujer,1,87,9201,Angol,De La Araucanía,U072,U00-U99,...,None,None,None,None,None,None,Resto de Chile,03-07-2020,Covid Sospechoso,2020
98110,485741,2020-05-06,Hombre,1,95,6115,Rengo,Del Libertador B. O'Higgins,U072,U00-U99,...,None,None,None,None,None,None,Resto de Chile,06-05-2020,Covid Sospechoso,2020


In [38]:
#----------------------------DEIS--------------------------------------------------------------------
s="DEIS"
n="DEIS_2020"
DEIS=pd.read_sql_table(n, con=cnx,schema=s)
DEIS=DEIS[DEIS['COVID'].notnull()]
#fallecidos nuevos regionales (DEIS)
fr=DEIS.groupby(['FECHA_DEF','Metropolitana'])['FECHA_STR'].count()
fr=fr.reset_index()
fr.columns=['Fecha','Region Metropolitana','Fallecidos Nuevos']
fr=fr.pivot(index='Fecha',columns=['Region Metropolitana'])
fr=fr.T.reset_index().T
fr.columns=fr.iloc[0].astype(str)+' '+fr.iloc[1].astype(str)+' (DEIS)'
fr=fr.iloc[3:].reset_index()
#fallecidos regional acumulado (deis)
fr=fr.fillna(0)
fra=fr[fr.columns.to_list()[1:]].cumsum()
fra.columns=[x.replace('Nuevos','Acumulados') for x in fra.columns]
#concatenar
fr=pd.concat([fr,fra],axis=1)

deis=DEIS.groupby('FECHA_DEF')['index'].count().reset_index()
deis.columns=['Fecha','Fallecidos DEIS Diarios']
deis=deis.merge(DEIS[DEIS['COVID']=='Covid PCR+'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (PCR+)'}),on='Fecha')
deis=deis.merge(DEIS[DEIS['COVID']=='Covid Sospechoso'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (Covid Sospechoso)'}),on='Fecha')
deis['Fallecidos DEIS (PCR+) Acumulado']=deis['Fallecidos DEIS Diarios (PCR+)'].cumsum()
deis['Fallecidos DEIS (Covid Sospechoso) Acumulado']=deis['Fallecidos DEIS Diarios (Covid Sospechoso)'].cumsum()
deis['Fallecidos DEIS Acumulado']=deis['Fallecidos DEIS Diarios'].cumsum()
deis['Fallecidos DEIS (PCR+) x Millón/Hab.']=deis['Fallecidos DEIS (PCR+) Acumulado']/(popTOT/1000000)
deis['Fallecidos DEIS (Covid Sospechoso) x Millón/Hab.']=deis['Fallecidos DEIS (Covid Sospechoso) Acumulado']/(popTOT/1000000)
deis['Fallecidos DEIS x Millón/Hab.']=deis['Fallecidos DEIS Acumulado']/(popTOT/1000000)


DEIS=deis.merge(fr,on='Fecha',how='left')
#----------------------------DEIS--------------------------------------------------------------------

In [52]:
#----------------------------ICOVID--------------------------------------------------------------------

#casos nuevos según fecha de inicio de síntomas (icovid)
s="producto53"
n="producto53_confirmados_"
nuevos=pd.read_sql_table(n, con=cnx,schema=s)
nuevos=pd.DataFrame(nuevos.groupby('fecha')['Promedio'].sum())
nuevos.columns=['Casos Nuevos (ICOVID)']
nuevos['Incidencia Nuevos (ICOVID) x 100 K Hab.']=nuevos['Casos Nuevos (ICOVID)']/popTOT*100000
nuevos['Casos Acumulados (ICOVID)']=nuevos['Casos Nuevos (ICOVID)'].cumsum()

#casos activos según fecha de inicio de síntomas (icovid)
s="producto53"
n="producto53_confirmados_regionale_casos_activos"
activos=pd.read_sql_table(n, con=cnx,schema=s)
activos=pd.DataFrame(activos.groupby('fecha')['Promedio'].sum())
activos.columns=['Casos Activos (ICOVID)']
activos['Incidencia Activos (ICOVID) x 100 K Hab.']=activos['Casos Activos (ICOVID)']/popTOT*100000

#activos regional icovid
s="producto53"
n="producto53_confirmados_regionale_casos_activos"
ar=pd.read_sql_table(n, con=cnx,schema=s)
ar=pd.DataFrame(ar.groupby(['fecha','Region_Metropolitana'])[['Poblacion','Promedio']].sum()).reset_index()
ar=ar.pivot(index='fecha',columns=['Region_Metropolitana']).T.reset_index().T
ar.columns=ar.iloc[0].astype(str)+' '+ar.iloc[1].astype(str)+' Casos Activos (ICOVID)'
ar=ar.iloc[3:].reset_index()
ar=ar.rename(columns={'fecha':'Fecha'})

#nuevos regionales icovid
s="producto53"
n="producto53_confirmados_regionale"
nr=pd.read_sql_table(n, con=cnx,schema=s)
nr=pd.DataFrame(nr.groupby(['fecha','Region_Metropolitana'])[['Poblacion','Promedio']].sum()).reset_index()
nr=nr.pivot(index='fecha',columns=['Region_Metropolitana']).T.reset_index().T
nr.columns=nr.iloc[0].astype(str)+' '+nr.iloc[1].astype(str)+' Casos Nuevos (ICOVID)'
nr=nr.iloc[3:].reset_index()
nr=nr.rename(columns={'fecha':'Fecha'})

#icovid consolidado
icovid=pd.concat([nuevos,activos],axis=1)
icovid=icovid.reset_index()
icovid=icovid.rename(columns={'fecha':'Fecha'})

icovid=icovid.merge(ar,how='left',on='Fecha').merge(nr,how='left',on='Fecha')
#----------------------------ICOVID--------------------------------------------------------------------

In [58]:
#----------------------------MINSAL--------------------------------------------------------------------

#casos MINSAL
s="producto5"
n="producto5_TotalesNacionales_T"
minsal=pd.read_sql_table(n, con=cnx,schema=s)
minsal=minsal.drop('index',axis=1)
minsal.columns=[x+' (MINSAL)' for x in minsal.columns]
minsal=minsal.rename(columns={'Fecha (MINSAL)': 'Fecha'})
minsal['Fecha']=pd.to_datetime(minsal['Fecha'],errors='coerce')

#casos MINSAL regionales
s="producto3"
n="producto3_TotalesPorRegion_std"
minsalr=pd.read_sql_table(n, con=cnx,schema=s)
#minsal=minsal.drop('index',axis=1)
#minsal.columns=[x+' (MINSAL)' for x in minsal.columns]
#minsal=minsal.rename(columns={'Fecha (MINSAL)': 'Fecha'})
#minsal['Fecha']=pd.to_datetime(minsal['Fecha'],errors='coerce')

#casos nuevos regional minsal
s="producto13"
n="producto13_CasosNuevosCumulativo_std"
nrm=pd.read_sql_table(n, con=cnx,schema=s)
nrm=nrm[nrm['Region']!='Total']
nrm=pd.DataFrame(nrm.groupby(['Fecha','Region_Metropolitana'])['Total'].sum()).reset_index()
nrm=nrm.pivot(index='Fecha',columns=['Region_Metropolitana']).T.reset_index().T
nrm.columns='Casos Nuevos (MINSAL) '+nrm.iloc[1].astype(str)
nrm=nrm.iloc[3:].reset_index()

#casos activos por fecha inicio de sintomas regional (minsal)
s="producto19"
n="producto19_CasosActivosPorComuna_std"
arms=pd.read_sql_table(n, con=cnx,schema=s)
arms=arms.fillna(0)
arms=pd.DataFrame(arms.groupby(['Fecha','Region_Metropolitana'])['Casos activos'].sum()).reset_index()
arms=arms.pivot(index='Fecha',columns=['Region_Metropolitana']).T.reset_index().T
arms.columns='Casos Activos (MINSAL) '+arms.iloc[1].astype(str)
arms=arms.iloc[3:].reset_index()

#activos regional minsal
s="producto25"
n="producto25_CasosActualesPorComuna_std"
arm=pd.read_sql_table(n, con=cnx,schema=s)
arm=arm[arm['Region']!='Total']
arm=pd.DataFrame(arm.groupby(['Fecha','Region_Metropolitana'])[['Casos actuales']].sum()).reset_index()
arm=arm.pivot(index='Fecha',columns=['Region_Metropolitana']).T.reset_index().T
arm.columns=arm.iloc[0].astype(str)+' '+arm.iloc[1].astype(str)+' (MINSAL)'
arm=arm.iloc[3:].reset_index()
#arm=arm.rename(columns={'fecha':'Fecha'})

minsal=minsal.merge(arm,how='left',on='Fecha').merge(nrm,how='left',on='Fecha').merge(arms,how='left',on='Fecha')
#----------------------------MINSAL--------------------------------------------------------------------

In [63]:
s='@perez'
n='nacional@perez'
nacional=pd.read_sql_table(n, con=cnx,schema=s)

In [66]:
nacional.sort_values(by='Fecha').dropna(subset=['Fecha'])

,index,Fecha,Test_PCR_notificados_acumulados,Laboratorios_que_reportan_en_esa_fecha,Confirmados_acumulados_totales__incluye_asintomáticos_,Confirmados_asintomáticos_acumulados_totales,Muertes_acumuladas_totales,Pacientes_actualmente_en_UCI,Pacientes_covid_positivo_actualmenteen_cama_básica,Pacientescovid_positivoactualmenteen_cama_media,...,Pacientes_sospechosos_acumulados_en_Epivigila,Confirmados_acumulados_ingresados_en_Epivigila,Pacientes_acumulados_cn_info_de_ingreso_a_hospital,Pacientes_acumulados_cn_info_de_ingreso_a_UTI/UCI,Confirmados_dossemanas_atrasmenos_muertoshasta_día_actual,Nuevos_confirmados_dos_semanas_atrás_menos_nuevos_muertos_hoy,Camas UCI Desocupadas,VMI Desocupadas,VMI no-COVID,UCI no-COVID
0,0,2020-03-06,None,nan,7.000,nan,0.000,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
1,1,2020-03-07,None,nan,9.000,nan,0.000,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
2,2,2020-03-08,None,nan,12.000,nan,0.000,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
3,3,2020-03-09,None,nan,17.000,nan,0.000,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
4,4,2020-03-10,None,nan,23.000,nan,0.000,nan,nan,nan,...,270.000,18.000,nan,None,None,None,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,204,2020-09-26,3228414,128.000,457848.000,63223.000,12640.000,888.000,986.000,895.000,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
205,205,2020-09-27,3265887,112.000,459617.000,63824.000,12697.000,884.000,994.000,890.000,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
206,206,2020-09-28,3294806,90.000,461247.000,64376.000,12724.000,865.000,1017.000,899.000,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
207,207,2020-09-29,3322315,127.000,462938.000,64825.000,12740.000,849.000,1043.000,897.000,...,nan,nan,nan,None,None,None,nan,nan,nan,nan


In [60]:
master=master.merge(icovid,on='Fecha',how='left').merge(minsal,on='Fecha',how='left')

In [61]:
master.sort_values(by='Fecha')

,Fecha,Fallecidos DEIS Diarios,Fallecidos DEIS Diarios (PCR+),Fallecidos DEIS Diarios (Covid Sospechoso),Fallecidos DEIS (PCR+) Acumulado,Fallecidos DEIS (Covid Sospechoso) Acumulado,Fallecidos DEIS Acumulado,Fallecidos DEIS (PCR+) x Millón/Hab.,Fallecidos DEIS (Covid Sospechoso) x Millón/Hab.,Fallecidos DEIS x Millón/Hab.,...,Casos activos confirmados (MINSAL)_y,Casos probables acumulados (MINSAL)_y,Casos activos probables (MINSAL)_y,Casos nuevos sin notificar (MINSAL)_y,Casos actuales Región Metropolitana (MINSAL),Casos actuales Resto de Chile (MINSAL),Casos Nuevos (MINSAL) Región Metropolitana,Casos Nuevos (MINSAL) Resto de Chile,Casos Activos (MINSAL) Región Metropolitana,Casos Activos (MINSAL) Resto de Chile
0,2020-03-20,2,1,1,1,1,2,0.051,0.051,0.103,...,nan,nan,nan,nan,NaN,NaN,57.000,35.000,NaN,NaN
1,2020-03-21,2,1,1,2,2,4,0.103,0.103,0.206,...,nan,nan,nan,nan,NaN,NaN,55.000,48.000,NaN,NaN
2,2020-03-26,2,1,1,3,3,6,0.154,0.154,0.308,...,nan,nan,nan,nan,NaN,NaN,64.000,100.000,NaN,NaN
3,2020-03-27,3,1,2,4,5,9,0.206,0.257,0.463,...,nan,nan,nan,nan,NaN,NaN,192.000,112.000,NaN,NaN
4,2020-03-28,5,1,4,5,9,14,0.257,0.463,0.719,...,nan,nan,nan,nan,NaN,NaN,146.000,153.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2020-10-03,63,49,14,13149,4610,17759,675.752,236.917,912.669,...,nan,61808.000,3512.000,91.000,NaN,NaN,321.000,1559.000,NaN,NaN
194,2020-10-04,57,35,22,13184,4632,17816,677.551,238.047,915.599,...,nan,62033.000,3443.000,62.000,NaN,NaN,357.000,1351.000,NaN,NaN
195,2020-10-05,63,44,19,13228,4651,17879,679.812,239.024,918.836,...,nan,62112.000,3172.000,102.000,8692.000,28246.000,363.000,1204.000,3356.000,4035.000
196,2020-10-06,53,31,22,13259,4673,17932,681.406,240.154,921.560,...,nan,62178.000,2833.000,35.000,NaN,NaN,242.000,1312.000,NaN,NaN


In [ ]:
name='consolidado_nacional'
schema='nacional'
if not cnx.dialect.has_schema(cnx, schema):
    print('schema '+schema+' does not exist, creating it')
    cnx.execute(sqlalchemy.schema.CreateSchema(schema))
else:
    print('schema '+schema+' exists, will not be created')
master.to_sql(name, schema=schema,con=cnx,if_exists='replace')